In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, MaxPooling1D, Conv2D, BatchNormalization
from keras.models import Model, Sequential
import numpy as np

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Keras is a great place to start, it has a relatively simple design, and uses TensorFlow under the hood. And -- it has the mnist data already available.


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train.shape

(60000, 28, 28)

We do a tiny bit of data prep -- normalization, meaning we divide by the max value to make all the pixes on the range of 0 - 1, this helps the model learn faster. For fun you can take out the ` / np.max(...)` and see how much longer it takes for the accuracy rise.

And -- we reshape. This is about meeting Keras expectations. The convolution layers are set up for 3D data -- meaning (x, y, color) channel pixels. Since the source mnist data is just (x, y), we have to shape the grey scale into the color channel position in the matrix, adding one additional dimension.

In [5]:
x_train = np.expand_dims(x_train / np.max(x_train), -1)
x_test = np.expand_dims(x_test / np.max(x_test), -1)
x_train.shape

(60000, 28, 28, 1)

For the output y labels, we need to convert the digit identifiers 0, 1, ... 8, 9 to one hot encodings where they are 10 slots, with a 0 or one acting as a flag.

In [6]:
y_train[0:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [7]:
train_labels = keras.utils.to_categorical(y_train, 10)
test_labels = keras.utils.to_categorical(y_test, 10)

train_labels[0:10]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

And now -- an actual deep network, this is a now classic design, using convolution, pooling, and dropout. Finally, the model ends in a dense layer with softmax -- seems familiar, this softmax output is just like our logistic regression. 

The difference here is -- we have created a deep learning model with many layers.

And now, using the Keras, build a model that has convolution, pooling, dropout and a final softmax classification.


One thing to note here is Flatten. Because our images are two dimensional *x,y* pairs, and our output is one dimension -- a class 0-9, Flatten is needed to reduce the dimensions.


In [8]:
input_shape = x_train[0].shape
num_classes = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

With the model assembled, we compile it, which prepares the model for execution with a solver. And then we fit it -- using the training data and labels to learn parameters, and the testing data and labels to check how well the model works.

This is an important point -- holding out part of the data to test. If you use all of you data in training, you can end up with a model that merely memorizes your input data, but cannot make predictions about new, unseen data. This is a phenomena known as *overfitting*.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, train_labels,
                    batch_size=64,
                    epochs=8,
                    verbose=1,
                    validation_data=(x_test, test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 166s 3ms/step - loss: 0.2081 - acc: 0.9368 - val_loss: 0.0510 - val_acc: 0.9833
Epoch 2/8
60000/60000 [==============================] - 168s 3ms/step - loss: 0.0818 - acc: 0.9752 - val_loss: 0.0363 - val_acc: 0.9880
Epoch 3/8
33088/60000 [===============>..............] - ETA: 1:11 - loss: 0.0634 - acc: 0.9807

And now we'll get a report as to how well we're classifying.

In [ ]:
import sklearn.metrics
predictions = model.predict_classes(x_test)
print(sklearn.metrics.classification_report(test_labels, predictions))